In [1]:
import os
import sys
!export PYTHONPATH="${PYTHONPATH}:~/gt/3D_tracking/DEV/Tracking/nuscenes"
import numpy as np

from sklearn.utils.linear_assignment_ import linear_assignment

from nuscenes import NuScenes
from nuscenes.eval.common.config import config_factory
from nuscenes.eval.tracking.evaluate import TrackingEval
from nuscenes.eval.detection.data_classes import DetectionConfig
from nuscenes.eval.detection.data_classes import DetectionBox
from nuscenes.eval.tracking.data_classes import TrackingBox
from nuscenes.eval.common.loaders import load_prediction, load_gt, filter_eval_boxes
from nuscenes.eval.tracking.loaders import create_tracks
from pyquaternion import Quaternion
#from main import iou3d, convert_3dbox_to_8corner
import argparse
import pickle 
from utils.dict import *
from utils.stats import add_center_dist
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt


In [2]:
training_file_name = "/media/liangxu/ArmyData/nuscenes/Tracking_result/tracking_tmp/training_data"
training_data = pickle.load(open(training_file_name , 'rb'))

## Testing the dataset

In [3]:
print("The Length of the dataset", len(training_data))

The Length of the dataset 3540


In [5]:
print("Elements in each track", len(training_data[0]))

Elements in each track 34


In [3]:
print(training_data[0][0].keys())

dict_keys(['sample_annotation', 'sample', 'sample_data_lidar', 'ego_pose', 'detection'])


In [4]:
training_data[0][0]['ego_pose']

{'token': '10bd27748c854ab0847780887130b85c',
 'timestamp': 1531883719448979,
 'rotation': [-0.12163871281229555,
  0.0031403839467553972,
  -0.0022023796453806243,
  -0.9925670310150003],
 'translation': [390.50662393327957, 1100.2516852821325, 0.0]}

In [10]:
training_data[0][0]['sample_annotation']

{'token': '02183ddb72974cd8b85780d2d0d4d1a6',
 'sample_token': '9e28820bd2ba4eb480e1b9079f17c30c',
 'instance_token': '1ef7286ee82e46d99e98d6d6d7c1d7ac',
 'visibility_token': '3',
 'attribute_tokens': ['58aa28b1c2a54dc88e169808c07331e3'],
 'translation': [358.212, 1081.729, 0.756],
 'size': [1.954, 5.078, 1.556],
 'rotation': [0.8342998112464631, 0.0, 0.0, 0.5513110056529944],
 'prev': '',
 'next': '798235a654374c01974861b40497e484',
 'num_lidar_pts': 1,
 'num_radar_pts': 0,
 'category_name': 'vehicle.car'}

In [11]:
training_data[0][0]['sample']

{'token': '9e28820bd2ba4eb480e1b9079f17c30c',
 'timestamp': 1531883719448979,
 'prev': '',
 'next': '3a9918ace7f3435ea3fe8a024cb3f181',
 'scene_token': 'c3e0e9f6ee8d4170a3d22a6179f1ca3a',
 'data': {'RADAR_FRONT': 'ec4889619f6946dbbe5a6397db78c785',
  'RADAR_FRONT_LEFT': 'a08f04817a7c4e028374a43190c547c1',
  'RADAR_FRONT_RIGHT': '8fa8c10c45104b36aed8d10ace890b18',
  'RADAR_BACK_LEFT': 'd86cb015480a448c9afd48df98881522',
  'RADAR_BACK_RIGHT': 'a8a3da37af1249d2be03bb76466d5c84',
  'LIDAR_TOP': '10bd27748c854ab0847780887130b85c',
  'CAM_FRONT': '8ed2a7ea9a894eac8b3cd43b706f39bc',
  'CAM_FRONT_RIGHT': '640f041fd567434180af197244f7c907',
  'CAM_BACK_RIGHT': '3974f911204c48ee8d768886bc1a70df',
  'CAM_BACK': '4a5e6fb26c364c4fa4fdbbb2406e3b2c',
  'CAM_BACK_LEFT': 'e55b082e9d154a8aac3effa77f79ac12',
  'CAM_FRONT_LEFT': '88381f260b5244948ca615bc2903025b'},
 'anns': ['cada68b62cb44a9fba02e46552323e53',
  '2bcb9a25a0fd4fef93d53aacae476c46',
  '7f4a7f5074984d0d953d30801086db08',
  '6a8c4a9c08e34f2aa

In [12]:
training_data[0][0]['sample_data_lidar']

{'token': '10bd27748c854ab0847780887130b85c',
 'sample_token': '9e28820bd2ba4eb480e1b9079f17c30c',
 'ego_pose_token': '10bd27748c854ab0847780887130b85c',
 'calibrated_sensor_token': 'efb2c9e1d36f41df89daf58cb67738f8',
 'timestamp': 1531883719448979,
 'fileformat': 'pcd',
 'is_key_frame': True,
 'height': 0,
 'width': 0,
 'filename': 'samples/LIDAR_TOP/n015-2018-07-18-11-07-57+0800__LIDAR_TOP__1531883719448979.pcd.bin',
 'prev': '',
 'next': '090474586bc24e13b37d978ae215a646',
 'sensor_modality': 'lidar',
 'channel': 'LIDAR_TOP'}

In [24]:
training_data[0][0]['detection'].attribute_name

'vehicle.moving'

In [33]:
count = 0
training_data_moving = []
for i in range(len(training_data)):
    if training_data[i][0]['detection'].attribute_name == 'vehicle.moving':
        count += 1
        training_data_moving.append(training_data[i])
    else: 
        print(training_data[i][0]['detection'].attribute_name)
print("Moving car Number ", count)

vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.pa

vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.parked
vehicle.pa

In [31]:
len(training_data_moving)

934

In [32]:
training_moving_file_name = "/media/liangxu/ArmyData/nuscenes/Tracking_result/tracking_tmp/training_data_moving"
with open(training_moving_file_name, 'wb') as f:
    pickle.dump(training_data_moving, f)
